In [1]:
spark.sql("""show tables in luzon_etl_s3""").show()

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
4,application_1602734624556_0005,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+--------------------+-----------+
|    database|           tableName|isTemporary|
+------------+--------------------+-----------+
|luzon_etl_s3|       t_admin_audit|      false|
|luzon_etl_s3|  t_auto_review_loan|      false|
|luzon_etl_s3|          t_bankcard|      false|
|luzon_etl_s3|   t_channel_details|      false|
|luzon_etl_s3|  t_collection_audit|      false|
|luzon_etl_s3|t_collection_blac...|      false|
|luzon_etl_s3|           t_contact|      false|
|luzon_etl_s3|          t_customer|      false|
|luzon_etl_s3|        t_employment|      false|
|luzon_etl_s3|t_engine_rule_detail|      false|
|luzon_etl_s3|          t_loan_app|      false|
|luzon_etl_s3|t_loan_app_review...|      false|
|luzon_etl_s3|t_loan_app_status...|      false|
|luzon_etl_s3|        t_loan_issue|      false|
|luzon_etl_s3|              t_lpay|      false|
|luzon_etl_s3|      t_lpay_deposit|      false|
|luzon_etl_s3|           t_message|      false|
|luzon_etl_s3|     t_personal_info|     

In [ ]:
col={
    "luzon": {
        "t_auto_review_loan": [
            "ktp",
            "mobile"
        ],
        "t_bankcard": [
            "card_no",
        ],
        "t_contact": [
            "mobile",
            "credential_no"
        ],
         "t_customer": [
            "mobile",
            "imei"
        ], 
        "t_employment": [
            "credential_no",
            "imei"
        ],
         "t_loan_app": [
            "credential_no"
        ], 
        "t_personal_info": [
            "credential_no",
            "email"
        ],
         "t_record_contact": [
            "mobile"
        ],
         "t_record_personal_info": [
            "credential_no"
        ]
    },
    "banda": {
        "t_personal_info": [
            "credential_no"
        ]
    }
}

In [1]:
spark.sql("""select  * from luzon_etl_s3.t_auto_review_loan """).printSchema()

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1602817165865_0002,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- etldate: string (nullable = true)
 |-- id: long (nullable = true)
 |-- riskengine_id: string (nullable = true)
 |-- customer_id: long (nullable = true)
 |-- loan_id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- ktp: string (nullable = true)
 |-- mobile: string (nullable = true)
 |-- success: string (nullable = true)
 |-- error_message: string (nullable = true)
 |-- result: string (nullable = true)
 |-- reject_message: string (nullable = true)
 |-- create_time: timestamp (nullable = true)
 |-- finish_time: timestamp (nullable = true)
 |-- duration: long (nullable = true)
 |-- real_policy: string (nullable = true)
 |-- switch_manual_reason: string (nullable = true)
 |-- raw_data: string (nullable = true)
 |-- year: string (nullable = true)

In [59]:
import sys
#notification导表到notification_etl_s3
def install():
    sc.uninstall_package('boto3')
    sc.install_pypi_package("boto3")
install()
from concurrent import futures
import numpy as np
from pyspark.sql import SparkSession
from datetime import datetime,timedelta
import pytz
import boto3
import json
import pytz
import hmac
import pyspark.sql.functions as F
from hashlib import sha256 
from pyspark.sql.functions import col
from pyspark.sql import types as T
from pyspark.sql.functions import lit
from pyspark.sql.types import StringType

# print(datetime.now(pytz.timezone("Asia/Shanghai")).strftime( '%Y-%m-%d'))
today=(datetime.now()+ timedelta(-1)).strftime( '%Y-%m-%d')
yesterday=(datetime.now(pytz.timezone("Asia/Shanghai"))+ timedelta(-1)).strftime( '%Y-%m-%d')
tablemap={"banda_stream_etl":"banda"} #,
# ,"banda_stream_etl":"1"
tablesql_1=" FROM  (   SELECT    *  , row_number() OVER (PARTITION BY id  ORDER BY date_format(etldate,'yyyy-MM-dd HH:mm:ss.SSS') DESC  , if(etlindex is null ,0,etlindex)  desc)  row_num FROM  "
tablesql_2=" ORDER BY id  ASC )  a  WHERE a.row_num = 1   AND a.kind <> 'delete' "
# 替换手机号
def get_secret_obj():
    df=spark.read.text("s3://rupiahplus-configs/etl/data_secrt/col.json").collect()
    keymap=''
    for  row in df:
        keymap=keymap+row['value']
    json_content=json.loads(keymap)
    return json_content
colmap= get_secret_obj()
print(colmap)
mobileType=colmap["mobileType"]
def getRelmobile(colNm,str):
    if colNm in mobileType:
        relmobile = str.strip().replace("+", "").replace("-", "").replace(" ", "").replace('"', '')
        if (relmobile.startswith("62")):
            relmobile = relmobile.replace("62", "0",1)
        if relmobile.startswith("0") == False:
            relmobile = "0" + relmobile;
        return relmobile
    return str
# hmac_256加密
def hmac_sha256(key,colNm,value):
    if value!=None:
#         处理手机号
        rtn=getRelmobile(colNm,value)
        h = hmac.new(key.encode('utf-8'),digestmod=sha256)
        h.update(rtn.encode('utf-8'))
        h_str = h.hexdigest()
        return h_str
    return ""
spark.udf.register("hmac_sha256",hmac_sha256,T.StringType())
def getTableColum(b):
    colum=""
    for index in range(len(b)):
        if(index>0 and index<len(b)-8):   
            colum=colum+setDef(b[index]["data_type"],b[index]["col_name"])+", "
    return colum[0:len(colum)-2]
def setDef(type,table_col):
    if(type[:7] == 'decimal'):
        return "ifnull("+table_col+",0) as "+table_col
    else:
        return table_col
def add_secret_col(dbType,tableNm,col):
    secret_Col=[]
    if colmap.get(dbType)!=None and colmap[dbType]!=None and colmap[dbType].get(tableNm)!=None:
        secret_Col=colmap[tablemap[databaseName]][tableNm]
    sh256Col =col
    hmac_key=colmap["hmac_key"]
#     sql="select hmac_sha256('{hmac_key}','123456789') as sh2".format(hmac_key=hmac_key)
    for i in secret_Col:
        sh256Col=sh256Col+ ", hmac_sha256('{hmac_key}','"+i+"', "+i+") "+ i+"_x"
    return sh256Col.format(hmac_key=hmac_key)
def make_col_temptable(databaseName,row):
    tableName=row["tableName"]
    if tableName=='t_personal_info' or tableName=='t_admin_group_rel' or tableName=='t_customer':
        tablecolum="desc " +databaseName+"."+tableName;
        tableSchema= spark.sql(tablecolum).collect()
        dbtype=tablemap[databaseName]
        tablepath="s3://rupiahplus-data-warehouse/aliyun/"+tablemap[databaseName]+"/"+row["tableName"]
        spark.catalog.refreshTable(databaseName+"."+tableName)
        secret_sql=add_secret_col(tablemap[databaseName],tableName,getTableColum(tableSchema))+", year"
        sql="select "+secret_sql+ tablesql_1+databaseName+"."+tableName+tablesql_2
#         print(sql)
        df=spark.sql(sql).drop("etlindex")
        if colmap.get(tablemap[databaseName]) and colmap[tablemap[databaseName]].get(tableName)!=None :
            for i in colmap[tablemap[databaseName]][tableName]:
                df=df.withColumn("col_x",F.col(i+"_x")).withColumn(i+"_x",F.col(i)).withColumn(i,F.col("col_x"))
        #     增加特殊列
        df=df.drop("col_x")
        if colmap["extraColumn"]!=None and colmap["extraColumn"].get(dbtype)!=None and colmap["extraColumn"][dbtype].get(tableName)!=None and len(colmap["extraColumn"][dbtype][tableName])>0:
            colmap["extraColumn"][dbtype][tableName].insert(0,"*")
            df=df.selectExpr(colmap["extraColumn"][dbtype][tableName])
            df=df.withColumn("year_new",F.col("year")).drop("year").withColumnRenamed("year_new","year")
        df.show()
#         
#         df.drop("col_x").write.mode("overwrite").partitionBy("year").orc(tablepath)
if __name__ == "builtins":
#     注册udf
    spark.udf.register("hmac_sha256",hmac_sha256,T.StringType())
    spark = SparkSession\
        .builder\
        .appName("Python Demo")\
        .config("hive.metastore.client.factory.class", "com.amazonaws.glue.catalog.metastore.AWSGlueDataCatalogHiveClientFactory") \
        .enableHiveSupport()\
        .getOrCreate()
#     spark.conf.set("spark.scheduler.mode","FAIR")
    spark.conf.set("spark.sql.shuffle.partitions",20)
    for databaseName in tablemap:
        print(databaseName)
        databasesql="show tables in "+databaseName
        tables=spark.sql(databasesql)
        tablelist=tables.collect();
        executor=None
        with futures.ThreadPoolExecutor(max_workers=10) as executor:
            futures_result=futures.wait([executor.submit(make_col_temptable,databaseName, tableNm) for tableNm in tablelist])
            for  future in futures_result[0]:
                print(future.result())

VBox()

Exception in thread cell_monitor-59:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/opt/conda/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in cell_monitor
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
  File "/opt/conda/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in <listcomp>
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
KeyError: 'jobGroup'



FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Found existing installation: boto3 1.16.24
Uninstalling boto3-1.16.24:
  Successfully uninstalled boto3-1.16.24

  Using cached boto3-1.16.24-py2.py3-none-any.whl (129 kB)

{'hmac_key': 'xUIAZ3grtPOxaPNK', 'mobileType': ['phone', 'phone_number', 'user_mobile', 'number', 'caller', 'callee', 'to', 'mobile_phone', 'mobile', 'backup_mobile'], 'extraColumn': {'banda': {'t_personal_info': ['cast(substring(credential_no_x, 11, 2) AS bigint) as birth_year', 'CASE WHEN cast(substring(credential_no_x, 11, 2) AS bigint) > 37 THEN (year(current_date()) - cast(substring(credential_no_x, 11, 2) AS bigint) - 1900) ELSE (year(current_date()) - cast(substring(credential_no_x, 11, 2) AS bigint) - 2000) END as age ']}}, 'luzon': {'t_auto_review_loan': ['ktp', 'mobile'], 't_bankcard': ['card_no'], 't_contact': ['mobile', 'credential_no'], 't_customer': ['mobile', 'imei'], 't_loan_app': ['credential_no'], 't_personal_info': ['credential_no', 'email'], 't_record_contact': ['mobile'], 't_record_personal_info

In [60]:
import sys
from concurrent import futures
import numpy as np
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from datetime import datetime,timedelta
import copy
import json
import pytz
import hmac
from hashlib import sha256 
from pyspark.sql.functions import col
from pyspark.sql.functions import lit
from pyspark.sql.types import StringType
from pyspark.sql import types as T
spark = SparkSession\
    .builder\
    .appName("Python Demo")\
    .config("hive.metastore.client.factory.class", "com.amazonaws.glue.catalog.metastore.AWSGlueDataCatalogHiveClientFactory") \
    .config("spark.driver.maxResultSize", "4g") \
    .enableHiveSupport()\
    .getOrCreate()
spark.conf.set("hive.exec.dynamic.partition.mode","nonstrict");
spark.conf.set("spark.sql.sources.partitionOverwriteMode", "dynamic")
spark.conf.set("spark.sql.shuffle.partitions",20)
spark.conf.set("spark.files.overwrite","true")
#     ,Apache Arrow：一个跨平台的在内存中以列式存储的数据层，用来加速大数据分析速度。其可以一次性传入更大块的数据，pyspark中已经有载入该模块，需要打开该设置：
spark.conf.set("spark.sql.execution.arrow.enabled", "true")
#     创建表时自动删除已存在的目录
spark.conf.set("spark.sql.legacy.allowCreatingManagedTableUsingNonemptyLocation","true")
dbmap={
    "banda":{
        "increment_database":"banda_stream_etl",
        "database":"banda-etl-s3",
        "realTime_path":"banda"
    }
}
nowdate=(datetime.now()).strftime('%Y-%m-%d')
yesterday=(datetime.now()+ timedelta(-1)).strftime('%Y-%m-%d')
sqltemp="""with {update_snapshot_table} as (
  select *
    from (
      SELECT * ,
         row_number() OVER (PARTITION BY id ORDER BY date_format(etldate,'yyyy-MM-dd HH:mm:ss.SSS') DESC, if(etlindex is NULL, 0, etlindex) desc) row_num
         FROM `{increment_database}`.`{tableNm}`
         where date(year || '-' || month || '-' || day) <=date('{nowdate}') and date(year || '-' || month || '-' || day) >= date('{yesterday}')
      and date_format(etldate,'yyyy-MM-dd HH:mm:ss.SSS') >= (SELECT max(date_format(etldate, 'yyyy-MM-dd HH:mm:ss.SSS')) from  `{database}`.`{tableNm}`) 
      )
    where row_num = 1
  )
select  {select_sql} 
from `{database}`.`{tableNm}` l
left join {update_snapshot_table}
on l.id = {update_snapshot_table}.id
where if({update_snapshot_table}.kind is not null,{update_snapshot_table}.kind,'') <> 'delete'
union
select {update_snapshot_sql} from (
  select *
  from `{increment_database}`.`{tableNm}`
  where
    date(year || '-' || month || '-' || day) <=date('{nowdate}') and date(year || '-' || month || '-' || day) >= date('{yesterday}')
    and date_format(etldate,'yyyy-MM-dd HH:mm:ss.SSS') >= (SELECT max(date_format(etldate, 'yyyy-MM-dd HH:mm:ss.SSS')) from  `{database}`.`{tableNm}`) 
    and kind = 'insert'
  ) new
left join {update_snapshot_table}
on new.id = {update_snapshot_table}.id where if({update_snapshot_table}.kind is not null,{update_snapshot_table}.kind,'') <> 'delete' """
def get_secret_obj():
    df=spark.read.text("s3://rupiahplus-configs/etl/data_secrt/col.json").collect()
    keymap=''
    for  row in df:
        keymap=keymap+row['value']
    json_content=json.loads(keymap)
    return json_content
colmap= get_secret_obj()   
hmac_key=colmap["hmac_key"]
mobileType=colmap["mobileType"]
def getRelmobile(colNm,str):
    if colNm in mobileType:
        relmobile = str.strip().replace("+", "").replace("-", "").replace(" ", "").replace('"', '')
        if (relmobile.startswith("62")):
            relmobile = relmobile.replace("62", "0",1)
        if relmobile.startswith("0") == False:
            relmobile = "0" + relmobile;
        return relmobile
    return str
# hmac_256加密
def hmac_sha256(key,colNm,value):
    if value!=None and value!='':
#         处理手机号
        rtn=getRelmobile(colNm,value)
        h = hmac.new(key.encode('utf-8'),digestmod=sha256)
        h.update(rtn.encode('utf-8'))
        h_str = h.hexdigest()
        return h_str
spark.udf.register("hmac_sha256",hmac_sha256,T.StringType())
def getTableColum(b,update_snapshot_table,dbtype,tableNm):
    colum=""
    snapshot_colum=""
    sqlstrtemp="if({update_snapshot_table}.id is null, {colNm}     , {snapshot_colNm})  {ascolNm} ,"
    sqlstr_secret_temp=" if({update_snapshot_table}.id is null, {colNm}     , hmac_sha256('{hmac_key}','{ascolNm}',{snapshot_colNm})) {ascolNm}_x , "
    for index in range(len(b)):
        if(index==1 or (index>2 and index<len(b)-7)):
#             print(index,b[index])
#             在year前面加上加密列
            if(index==len(b)-8) and colmap[dbtype]!=None and colmap[dbtype].get(tableNm)!=None:
                for secret_col in colmap[dbtype][tableNm]:
                    sqlstr=copy.copy(sqlstr_secret_temp)
                    colum=colum+sqlstr.format(colNm=setDef("string",secret_col,False),snapshot_colNm=setDef("string",secret_col,True),ascolNm=secret_col,update_snapshot_table=update_snapshot_table,hmac_key=hmac_key)
                    snapshot_colum=snapshot_colum+setsecret_SnapshotDef(secret_col).format(update_snapshot_table=update_snapshot_table,hmac_key=hmac_key)+" ,"
            sqlstr=copy.copy(sqlstrtemp)
            colum=colum+sqlstr.format(colNm=isX(dbtype,tableNm,setDef(b[index]["data_type"],b[index]["col_name"],False)),snapshot_colNm=setDef(b[index]["data_type"],b[index]["col_name"],True),ascolNm=b[index]["col_name"],update_snapshot_table=update_snapshot_table)
            snapshot_colum=snapshot_colum+setSnapshotDef(b[index]["data_type"],b[index]["col_name"]).format(update_snapshot_table=update_snapshot_table)+" ,"
    return colum[0:len(colum)-2],snapshot_colum[0:len(snapshot_colum)-2]
def isX(type,table,column):
    columnList=column.split(".")
    if len(columnList)>0 and colmap[type].get(table)!=None and columnList[len(columnList)-1] in colmap[type][table]:
        return column+"_x"
    return column
def setDef(type,table_col,is_snapshot):
    table_col_x=copy.copy(table_col)
    if(type[:7] == 'decimal'):
        if is_snapshot:
            return "ifnull({update_snapshot_table}."+table_col_x+",0)  "
        else:
            return "ifnull(l."+table_col_x+",0) "
    else:
        if is_snapshot:
            return "{update_snapshot_table}."+table_col_x
        else:
            return "l."+table_col_x
def setSnapshotDef(type,table_col):
    table_col_x=copy.copy(table_col)
    if(type[:7] == 'decimal'):
        return "ifnull({update_snapshot_table}."+table_col_x+",0) "+table_col
    else:
        return "{update_snapshot_table}."+table_col_x
def setsecret_SnapshotDef(table_col):
    table_col_x=copy.copy(table_col)
#     print("hmac_sha256('{hmac_key}','"+table_col_x+"', {update_snapshot_table}."+table_col_x+") "+ table_col_x+"_x")
    return "hmac_sha256('{hmac_key}','"+table_col_x+"', {update_snapshot_table}."+table_col_x+") "+ table_col_x+"_x"
#     return "sha2({update_snapshot_table}."+table_col+",256) " +table_col+"_x"
def execute(tablerow,dbtype):
#     print(tablerow,dbtype)
    tableName=tablerow["tableName"]
    if tableName=='t_personal_info' or tableName=='t_admin_group_rel' or tableName=='t_customer':
        update_snapshot_table="update_snapshot_"+dbtype+"_"+tableName
        spark.catalog.refreshTable("`"+dbmap[dbtype]["database"]+"`."+tableName)
        col=spark.sql("desc `"+dbmap[dbtype]["increment_database"]+"`."+tableName).collect()
        #     #真正的列,增量的列
        real_col,snapshot_col=getTableColum(col,update_snapshot_table,dbtype,tableName)
        sql=copy.copy(sqltemp)
        real_sql=sql.format(increment_database=dbmap[dbtype]["increment_database"],tableNm=tableName,nowdate=nowdate,select_sql=real_col.format(update_snapshot_table=update_snapshot_table),update_snapshot_sql=snapshot_col.format(update_snapshot_table=update_snapshot_table),database=dbmap[dbtype]["database"],yesterday=yesterday,update_snapshot_table=update_snapshot_table,hmac_key=hmac_key)
        df=spark.sql(real_sql).drop("etlindex")
        if colmap[dbtype].get(tableName)!=None :
            for i in colmap[dbtype][tableName]:
                df=df.withColumn("col_x",F.col(i+"_x")).withColumn(i+"_x",F.col(i)).withColumn(i,F.col("col_x"))
        df=df.drop("col_x")
        if colmap["extraColumn"]!=None and colmap["extraColumn"].get(dbtype)!=None and colmap["extraColumn"][dbtype].get(tableName)!=None and len(colmap["extraColumn"][dbtype][tableName])>0:
            colmap["extraColumn"][dbtype][tableName].insert(0,"*")
            df=df.selectExpr(colmap["extraColumn"][dbtype][tableName])
            df=df.withColumn("year_new",F.col("year")).drop("year").withColumnRenamed("year_new","year")
        df.show()
    #     df.drop("col_x").write.mode("overwrite").partitionBy("year").format("orc").saveAsTable("etl_s3_temp."+dbtype+"_"+tableName);
    #     tablepath='s3://rupiahplus-data-warehouse/aliyun/'+dbmap[dbtype]["realTime_path"]+'/'+tableName+"/"
    #     spark.sql("select  * from  "+"etl_s3_temp."+dbtype+"_"+tableName).write.mode("overwrite").partitionBy("year").orc(tablepath)
if __name__ == "builtins":
    print("start-----",datetime.now())
    for dbtype in dbmap:
        databasesql="show tables in "+dbmap[dbtype]["increment_database"]
        tables=spark.sql(databasesql)
        tablelist=tables.collect();
    #         for row in tablelist:
        executor=None
        with futures.ThreadPoolExecutor(max_workers=10) as executor:
            futures_result=futures.wait([executor.submit(execute, table,dbtype) for table in tablelist])
            for  future in futures_result[0]:
                print(future.result())
                print("end-----",datetime.now())

VBox()

Exception in thread cell_monitor-60:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/opt/conda/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in cell_monitor
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
  File "/opt/conda/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in <listcomp>
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
KeyError: 'jobGroup'



FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

start----- 2020-11-24 08:44:20.209004
+-------------------+--------+--------+------+----+
|            etldate|admin_id|group_id|    id|year|
+-------------------+--------+--------+------+----+
|2020-08-01 23:59:59|  101485|  100076|100965|2020|
|2020-08-01 23:59:59|  101529|    7006|101024|2020|
|2020-08-01 23:59:59|  101534|    7006|101029|2020|
|2020-08-01 23:59:59|  101367|  100004|101031|2020|
|2020-08-01 23:59:59|  101539|  100075|101037|2020|
|2020-08-01 23:59:59|  101548|  100076|101046|2020|
|2020-08-01 23:59:59|  101551|  100077|101049|2020|
|2020-08-01 23:59:59|  101554|  100077|101052|2020|
|2020-08-01 23:59:59|  101791|  100085|101582|2020|
|2020-08-01 23:59:59|  101552|  100002|101077|2020|
|2020-08-01 23:59:59|  100097|    7004|101098|2020|
|2020-08-01 23:59:59|  100931|  100002|100790|2020|
|2020-08-01 23:59:59|  100879|  100048|100810|2020|
|2020-08-01 23:59:59|  100881|  100050|100811|2020|
|2020-08-01 23:59:59|  101072|  100004|100856|2020|
|2020-08-01 23:59:59|  101

In [1]:
import sys
def install():
    sc.uninstall_package('boto3')
    sc.install_pypi_package("boto3")
install()
import hmac
from hashlib import sha256 
from pyspark.sql.functions import udf
from pyspark.sql.functions import lit
from pyspark.sql import types as T
import pyspark.sql.functions as F
from pyspark.sql.column import Column
import boto3
import json
def get_secret_obj():
    df=spark.read.text("s3://rupiahplus-configs/etl/data_secrt/col.json").collect()
    keymap=''
    for  row in df:
        keymap=keymap+row['value']
    json_content=json.loads(keymap)
    return json_content
colmap= get_secret_obj()
def hmac_sha256(key,value):
    if value!=None:
        h = hmac.new(key.encode('utf-8'),digestmod=sha256)
        h.update(str(value).encode('utf-8'))
        h_str = h.hexdigest()
        return h_str
spark.udf.register("hmac_sha256",hmac_sha256,T.StringType())
hmac_sha256=udf(hmac_sha256,returnType=T.StringType())

hmac_key=colmap["hmac_key"]
sql="select hmac_sha256('{hmac_key}','123456789') as sh2,'123456789' as num ".format(hmac_key=hmac_key)
df=spark.sql(sql)

df.withColumn('num',hmac_sha256(lit(hmac_key),F.col('num'))).show(1,False)




VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
6,application_1603851421330_0007,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…


  Using cached boto3-1.16.6-py2.py3-none-any.whl (129 kB)
  Using cached botocore-1.19.6-py2.py3-none-any.whl (6.7 MB)
  Using cached s3transfer-0.3.3-py2.py3-none-any.whl (69 kB)
  Using cached urllib3-1.25.11-py2.py3-none-any.whl (127 kB)
  Using cached python_dateutil-2.8.1-py2.py3-none-any.whl (227 kB)

{'hmac_key': 'xUIAZ3grtPOxaPNK', 'luzon': {'t_auto_review_loan': ['ktp', 'mobile'], 't_bankcard': ['card_no'], 't_contact': ['mobile', 'credential_no'], 't_customer': ['mobile', 'imei'], 't_loan_app': ['credential_no'], 't_personal_info': ['credential_no', 'email'], 't_record_contact': ['mobile'], 't_record_personal_info': ['credential_no']}, 'banda': {'t_personal_info': ['credential_no']}}
+----------------------------------------------------------------+----------------------------------------------------------------+
|sh2                                                             |num                                                             |
+-------------------------------

In [6]:
# def get_secret_obj():
#     s3 = boto3.resource('s3')
#     # s3://rupiahplus-configs/etl/data_secrt/format.1602818198787.json
#     content_object = s3.Object('rupiahplus-configs', 'etl/data_secrt/col.json')
#     file_content = content_object.get()['Body'].read().decode('utf-8')
#     json_content = json.loads(file_content)
#     print(json_content) 
#     return json_content
# import json
# df=spark.read.text("s3://rupiahplus-configs/etl/data_secrt/col.json").collect()
# keymap=''
# for  row in df:
#     keymap=keymap+row['value']
# json_content=json.loads(keymap)
# print(json_content['hmac_key']) 
table_col='name'
print("hmac_sha256('{hmac_key}','"+table_col+"', {update_snapshot_table}."+table_col+") "+ table_col+"_x")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

hmac_sha256('{hmac_key}','name', {update_snapshot_table}.name) name_x

In [20]:
# spark.sql("""
# select * from luzon_stream_etl.t_contact where id =752448
# """).show()
import datetime
print(datetime.datetime.now())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2020-11-09 10:16:28.554928

In [12]:
spark.sql("""

SELECT id,
credential_no_x,
         cast(substring(credential_no_x,
         11,
         2) AS bigint) birth_year,
        
    CASE
    WHEN cast(substring(credential_no_x, 11, 2) AS bigint) > 37 THEN
    (year(current_date()) - cast(substring(credential_no_x, 11, 2) AS bigint) - 1900)
    ELSE (year(current_date()) - cast(substring(credential_no_x, 11, 2) AS bigint) - 2000)
    END age,
    current_date()
FROM `banda-etl-s3`.t_personal_info
WHERE id=7544402
""").show(1,False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+----------------+----------+---+--------------+
|id     |credential_no_x |birth_year|age|current_date()|
+-------+----------------+----------+---+--------------+
|7544402|3213032603900002|90        |30 |2020-11-23    |
+-------+----------------+----------+---+--------------+

In [29]:
colnum={
        "extraColumn": {
        "banda": {
            "t_personal_info": 
                {
                    "birth_year": "cast(substring(credential_no_x, 11, 2) AS bigint)",
                    "age": "CASE WHEN cast(substring(credential_no_x, 11, 2) AS bigint) > 37 THEN (year(current_date()) - cast(substring(credential_no_x, 11, 2) AS bigint) - 1900) ELSE (year(current_date()) - cast(substring(credential_no_x, 11, 2) AS bigint) - 2000) END "
                }
        }
    },
}
#     print(tableNm1)
#     a=[i, tableNm1[i]  for i in tableNm1.keys()]
#     print(a)    
print(colnum["extraColumn"]['banda']["t_personal_info"])
for i in colnum["extraColumn"]['banda']["t_personal_info"]:
    print(i)



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{'birth_year': 'cast(substring(credential_no_x, 11, 2) AS bigint)', 'age': 'CASE WHEN cast(substring(credential_no_x, 11, 2) AS bigint) > 37 THEN (year(current_date()) - cast(substring(credential_no_x, 11, 2) AS bigint) - 1900) ELSE (year(current_date()) - cast(substring(credential_no_x, 11, 2) AS bigint) - 2000) END '}
birth_year
age